In [24]:
import numpy as np
import pandas as pd

table1 = pd.read_excel('table1.xlsx')
col1 = table1.columns
t1 = np.array(table1)

table2 = pd.read_excel('table2.xlsx')
col2 = table2.columns
t2 = np.array(table2)

table3 = pd.read_excel('table3.xlsx')
col3 = table3.columns
t3 = np.array(table3)


def interpol2_3(d_e, v_f):
    def find6i(d_e):
        if col2.contains(d_e):
            a = (col2 == d_e).argmax()
            return (a,a,1)
        else:
            a = (col2 > d_e).argmax()
            kf = (d_e - col2[a-1]) / (col2[a] - col2[a-1])
            return (a-1, a, kf)

    def find6j(v_f):
        row1 = t2[:,0]
        if (row1 == v_f).max() == True:
            a = (row1 == v_f).argmax() + 2
            return (a, a, 1)
        else:
            a = (row1 > v_f).argmax() + 2
            kf = (v_f - row1[a-3]) / (row1[a] - row1[a-3])
            return (a-3, a, kf)

    i_1, i_2, ki = find6i(d_e)
    j_1, j_2, kj = find6j(v_f)
    A = (1 - ki) * (1 - kj) * t2[j_1,i_1]
    B = (1 - kj) * ki * t2[j_1,i_2]
    C = kj * (1 - ki) * t2[j_2,i_1]
    D = ki * kj * t2[j_2,i_2]
    print(t2[j_1,i_1],t2[j_1,i_2],t2[j_2,i_1],t2[j_2,i_2])
    return A + B + C + D


def interpol3(K, v_f):
    def find7i(K):
        if col3.contains(K):
            a = (col3 == K).argmax()
            return (a,a,1)
        else:
            a = (col3 > K).argmax()
            kf = (K - col3[a-1]) / (col3[a] - col3[a-1])
            return (a-1, a, kf)
    def find7j(v_f):
        row1 = t3[:,0]
        if (row1 == v_f).max() == True:
            a = (row1 == v_f).argmax()
            return (a,a,1)
        else:
            a = (row1 > v_f).argmax()
            kf = (v_f - row1[a-1]) / (row1[a] - row1[a-1])
            return (a-1,a, kf)
    i_1, i_2, ki = find7i(K)
    j_1, j_2, kj = find7j(v_f)
    A = (1 - ki) * (1 - kj) * t3[j_1,i_1]
    B = (1 - kj) * ki * t3[j_1,i_2]
    C = kj * (1 - ki) * t3[j_2,i_1]
    D = ki * kj * t3[j_2,i_2]
    return A + B + C + D


def sector_calculation(t_v, c, l, v, K):
    F = c / (3600 * v) # 1
    near_F = (abs(t1[:,2]-F)).argmin() # 2
    a, b, F_n = t1[near_F] # 3
    v_f = c / (3600 * F_n) # 4
    d_e = 2*a*b/(a+b) # 5
    R = interpol2_3(d_e, v_f) # 6
    B = interpol3(K, v_f) # 7
    Ptr = l * R * B # 8
    Pt = 1.2 * 293 / (273+t_v) # 9
    Pg = Pt * (v_f ** 2) / 2 # 10
    return [a, b, d_e, F, F_n, v_f, R, B, Ptr, Pt, Pg]

def sector_calculation_with_ab(t_v, c, l, v, K, a, b):
    F_n = a*b/ 10**6
    v_f = c / (3600 * F_n) # 4
    d_e = 2*a*b/(a+b) # 5
    R = interpol2_3(d_e, v_f) # 6
    B = interpol3(K, v_f) # 7
    Ptr = l * R * B # 8
    Pt = 1.2 * 293 / (273+t_v) # 9
    Pg = Pt * (v_f ** 2) / 2 # 10
    return [c, l, a, b, d_e, F_n, v_f, R, B, Ptr, 0, 0, Pg]

answ_columns = ['c','l','a', 'b', 'd_e', 'F_n', 'v_f', 'R', 'B', 'P_tr', '-','-', 'P_g']
answers = []
i = 0
while True:
    sector = str(input('t_v | c | l | v | K | a | b >>'))
    if (sector == '0'):
        break
    sec = sector.split()
    t_v = float(sec[0])
    c = float(sec[1])
    l = float(sec[2])
    v = float(sec[3])
    K = float(sec[4])
    a = float(sec[5])
    b = float(sec[6])
    
#     t_v = float(input('t_v >> '))  # температура
#     if (t_v < 0):
#         break
#     c = float(input('c >> '))  # расход
#     if (c < 0):
#         break
#     l = float(input('l >> '))  # длина
#     v = float(input('v >> '))  # скорость
#     K = float(input('k >> '))  # абсолютная шероховатость
#     a = float(input('a >> '))
#     b = float(input('b >> '))
    sector1 = sector_calculation_with_ab(t_v, c, l, v, K, a, b)
    answers += [sector1]
    i += 1
    print(i,"-----\n")

n = len(answers)
    
df = pd.DataFrame(data=answers, columns=answ_columns, index=np.arange(1,n+1))
df.to_excel('answer.xlsx')

df

t_v | c | l | v | K | a | b >>18 177 3.3 1 1 320 150
0.083 0.0762 0.122 0.105
1 -----

t_v | c | l | v | K | a | b >>18 177 3.3 1 1 320 150
0.083 0.0762 0.122 0.105
2 -----

t_v | c | l | v | K | a | b >>0


,c,l,a,b,d_e,F_n,v_f,R,B,P_tr,-,-,P_g
1,177.0,3.3,320.0,150.0,204.255319,0.048,1.024306,0.086371,1.111896,0.316918,0,0,0.633848
2,177.0,3.3,320.0,150.0,204.255319,0.048,1.024306,0.086371,1.111896,0.316918,0,0,0.633848


In [16]:
2.*320.*150./(320.+150.)

204.25531914893617

In [2]:
kms_otvod_table = pd.read_excel('otvod.xlsx')
col_kms_otvod = kms_otvod_table.columns
kms_otvod_tab = np.array(kms_otvod_table)
def kms_otvod(a, b, angle):
    if (angle != 90 and angle != 45):
        print('pizdec')
        return
    row1 = kms_otvod_tab[:,0]
    a_i = (row1 >= a).argmax()
    b_i = (col_kms_otvod >= b).argmax()
    if (angle == 90):
        return kms_otvod_tab[a_i, b_i]
    if (angle == 45):
        return kms_otvod_tab[a_i + 1, b_i]    

In [3]:
kms_nagn_tr_pr_table = pd.read_excel('nagn_tr_pr.xlsx')
col_kms_nagn_tr_pr = kms_nagn_tr_pr_table.columns
kms_nagn_tr_pr_tab = np.array(kms_nagn_tr_pr_table)
def kms_nagn_tr_pr(i,j): # i - расх отв/ствол j - пл прох/ствол
    row1 = kms_nagn_tr_pr_tab[:,0]
    i_ind = (abs(row1-i-0.000001)).argmin()
    j_ind = (abs(col_kms_nagn_tr_pr-j-0.01)).argmin()
    return kms_nagn_tr_pr_tab[i_ind, j_ind]

In [4]:
kms_nagn_tr_otv_table = pd.read_excel('nagn_tr_otv.xlsx')
col_kms_nagn_tr_otv = kms_nagn_tr_otv_table.columns
kms_nagn_tr_otv_tab = np.array(kms_nagn_tr_otv_table)
def kms_nagn_tr_otv(i,j): # i - расх отв/ствол j - пл отв/ствол
    row1 = kms_nagn_tr_otv_tab[:,0]
    i_ind = (abs(row1-i-0.000001)).argmin()
    j_ind = (abs(col_kms_nagn_tr_otv-j-0.01)).argmin()
    return kms_nagn_tr_otv_tab[i_ind, j_ind]

In [5]:
kms_vs_tr_pr_table = pd.read_excel('vs_tr_pr.xlsx')
col_kms_vs_tr_pr = kms_vs_tr_pr_table.columns
kms_vs_tr_pr_tab = np.array(kms_vs_tr_pr_table)
def kms_vs_tr_pr(i, j, k): # i - пл прох/ствол j - расх отв/ствол k - пл отв/ствол
    row1 = kms_vs_tr_pr_tab[:,0]
    row2 = kms_vs_tr_pr_tab[:,1]
    i_ind = (abs(row1-i-0.0000001)).argmin()
    j_ind = (abs(row2-j-0.0000001)).argmin()
    k_ind = (abs(col_kms_vs_tr_pr-k-0.0000001)).argmin()
    return kms_vs_tr_pr_tab[i_ind + j_ind, k_ind]

In [130]:
kms_vs_tr_otv_table = pd.read_excel('vs_tr_otv.xlsx')
col_kms_vs_tr_otv = kms_vs_tr_otv_table.columns
kms_vs_tr_otv_tab = np.array(kms_vs_tr_otv_table)
def kms_vs_tr_otv(i, j, k): # i - пл прох/ствол j - расх отв/ствол k - пл отв/ствол
    row1 = kms_vs_tr_otv_tab[:,0]
    row2 = kms_vs_tr_otv_tab[:,1]
    i_ind = (abs(row1-i-0.0000001)).argmin()
    j_ind = (abs(row2-j-0.0000001)).argmin()
    k_ind = (abs(col_kms_vs_tr_otv-k-0.0000001)).argmin()
    return kms_vs_tr_otv_tab[i_ind + j_ind, k_ind]

,10.0,9.0,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1
0,1.0,0.05,-47.8,-70.00,76.60,-72.90,-63.10,-49.60,-33.8,-16.8
1,1.0,0.10,-23.6,-22.00,-20.00,-16.90,-13.70,-10.20,-6.6,-2.9
2,1.0,0.20,-4.7,-4.00,-3.00,-2.50,-1.75,-1.00,-0.35,0.15
3,1.0,0.30,-1.1,-0.80,-0.40,-0.15,0.15,0.40,0.55,0.6
4,1.0,0.40,0,0.20,0.40,0.55,0.70,0.75,0.8,0.7
5,1.0,0.50,0.5,0.60,0.70,0.80,0.90,0.90,0.9,0.7
6,1.0,0.60,0.7,0.80,0.85,0.90,0.95,0.95,0.9,0.65
7,0.9,0.05,NaN,-67.60,-76.30,-67.90,-58.30,-54.00,-30.8,-15.1
8,0.9,0.10,-21,-19.80,-17.90,-15.20,-12.30,-10.30,-5.8,-3.9
9,0.9,0.20,4,-3.40,-2.80,-2.10,-1.40,-0.70,-0.1,0.3
